In [ ]:
import pandas as pd

In [ ]:
# unicef data WH
wh_url = "https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/"
dataflow = "TRANSMONEE/"

In [ ]:
# indicators pulled
ind_list = [
    "EDUNF_OFST_L1T3",
    "EDUNF_ROFST_L1T3",
    "EDUNF_SAP_L1T3",
    "EDUNF_OFST_L1",
    "EDUNF_OFST_L2",
    "EDUNF_OFST_L3",
    "EDUNF_NER_L02",
    "EDUNF_SAP_L02",
    "EDUNF_CR_L2",
    "EDUNF_SAP_L2",
]

In [ ]:
# query SDMX with pandas
query_key = ".{}....".format("+".join(ind_list))
df = pd.read_csv(wh_url + dataflow + query_key + "?format=csv")

In [ ]:
df.columns

In [ ]:
# set numerator and denominator (OOS children average in selected countries by range of time, sex TOTAL)
numerator = ["EDUNF_OFST_L1T3"]
denominator = ["EDUNF_SAP_L1T3"]
countries = ["Armenia", "Azerbaijan", "Georgia"]
time = list(range(2008,2021))
total_code = ["_T"]

In [ ]:
query = (
        "INDICATOR in @ind_to_query & TIME_PERIOD in @time & `Geographic area` in @countries & SEX in @total_code \
& RESIDENCE in @total_code & WEALTH_QUINTILE in @total_code"
    )

In [ ]:
ind_to_query = numerator
numerators_df = df.query(query).groupby(
    [
        "INDICATOR",
        "Indicator",
        "Geographic area"
    ]
).agg(
    {"TIME_PERIOD": "last", "OBS_VALUE": "last"}
).reset_index().set_index(["Geographic area", "TIME_PERIOD"])
numerators_df

In [ ]:
# freaking out with pandas MULTIINDEX =0
numerators_df.index

In [ ]:
# select the denominators and set them with numerator compound-index
ind_to_query = denominator
denominators_df = df.query(query).set_index(["Geographic area", "TIME_PERIOD"])
# freaking out with pandas MULTIINDEX =0
denominators_df.index

In [ ]:
# match denominators and numerators compound-index
den_match_df = denominators_df[
    denominators_df.index.isin(numerators_df.index)
][["INDICATOR", "Indicator", "OBS_VALUE"]]
den_match_df

In [ ]:
# join numerators denominators with nan drops (missmatches)
num_den_not_nan = numerators_df.merge(
    den_match_df, on=["Geographic area", "TIME_PERIOD"], how="left", sort=False
).dropna(subset=["OBS_VALUE_y"])

In [ ]:
# calculation with join
round(num_den_not_nan["OBS_VALUE_x"].sum()/num_den_not_nan["OBS_VALUE_y"].sum()*100,1)

In [ ]:
# freaking out with pandas MULTIINDEX =0
ind_intersect = denominators_df.index.intersection(numerators_df.index)
ind_intersect

In [ ]:
# freaking out with pandas MULTIINDEX =0
ind_intersect = numerators_df.index.intersection(denominators_df.index)
ind_intersect

In [ ]:
# calculation without join
round(numerators_df.loc[ind_intersect]["OBS_VALUE"].sum()/denominators_df.loc[ind_intersect]["OBS_VALUE"].sum()*100,1)

In [ ]:
# calculation with a "pack" of numerators
numerator = ["EDUNF_OFST_L1", "EDUNF_OFST_L2", "EDUNF_OFST_L3"]
# what about same algorithm without a pack? --> works SUPERB
numerator = ["EDUNF_OFST_L1T3"]
ind_to_query = numerator
numerators_df = df.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

In [ ]:
# freaking out with pandas MULTIINDEX =0
numerator_pairs.index

In [ ]:
# order in intersection should be conmutative
ind_intersect = numerator_pairs.index.intersection(denominators_df.index)
ind_intersect

In [ ]:
# regional average using "pack" of numerators
# BTW: does this work for a pack of denominators?
# do we have "pack" of denominators case?
round(numerator_pairs.loc[ind_intersect]["OBS_VALUE"].sum() / 
      denominators_df.loc[ind_intersect]["OBS_VALUE"].sum()*100,1)

In [ ]:
absolute = True
denominators = denominators_df.loc[ind_intersect]["OBS_VALUE"]
indicator_sum = (
    numerator_pairs.loc[ind_intersect]["OBS_VALUE"].to_numpy().sum(
    ) / denominators.to_numpy().sum()
    if absolute
    else (
        numerator_pairs["OBS_VALUE"] * denominators / denominators.to_numpy().sum()
    ).dropna().to_numpy().sum()
)
# will drop missing countires
round(indicator_sum*100,1)

In [ ]:
# check the call with rates! (and only one numerator in the list)
numerator = ["EDUNF_NER_L02"]
ind_to_query = numerator
numerators_df = df.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

In [ ]:
denominator = ["EDUNF_SAP_L02"]
# select the denominators and set them with numerator compound-index
ind_to_query = denominator
denominators_df = df.query(query).set_index(["Geographic area", "TIME_PERIOD"])

In [ ]:
# select only those denominators that match avalible indicators
ind_intersect = numerator_pairs.index.intersection(
    denominators_df.index)

In [ ]:
absolute = False
denominators = denominators_df.loc[ind_intersect]["OBS_VALUE"]
indicator_sum = (
    numerator_pairs.loc[ind_intersect]["OBS_VALUE"].to_numpy().sum(
    ) / denominators.to_numpy().sum()
    if absolute
    else (
        numerator_pairs["OBS_VALUE"] / 100 * denominators / denominators.to_numpy().sum()
    ).dropna().to_numpy().sum()
)
# will drop missing countires
round(indicator_sum*100,1)

In [ ]:
# check it out BETO!
denominators

In [ ]:
(35.14269/100*129438.0 + 35.36377/100*511204.0 + 40.53383/100*137347.0) / (129438.0 + 511204.0 + 137347.0) * 100

In [ ]:
# check the call with rates! (and only one numerator in the list)
numerator = ["EDUNF_CR_L2"]
ind_to_query = numerator
numerators_df = df.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

In [ ]:
denominator = ["EDUNF_SAP_L2"]
# select the denominators and set them with numerator compound-index
ind_to_query = denominator
denominators_df = df.query(query).set_index(["Geographic area", "TIME_PERIOD"])

In [ ]:
# select only those denominators that match avalible indicators
ind_intersect = numerator_pairs.index.intersection(
    denominators_df.index)

In [ ]:
absolute = False
denominators = denominators_df.loc[ind_intersect]["OBS_VALUE"]
indicator_sum = (
    numerator_pairs.loc[ind_intersect]["OBS_VALUE"].to_numpy().sum(
    ) / denominators.to_numpy().sum()
    if absolute
    else (
        numerator_pairs["OBS_VALUE"] / 100 * denominators / denominators.to_numpy().sum()
    ).dropna().to_numpy().sum()
)
# will drop missing countires
round(indicator_sum*100,4)

In [ ]:
# check it out BETO!
denominators

In [ ]:
(96.96779*173593.0 + 98.65000*142350.0) / (173593.0 + 142350.0)